# Capitulo 15: Agrupando Datos.

#### Ejemplos.

##### Agrupación básica

##### Agrupando en una columna.
Usando el siguiente DataFrame

In [6]:
import pandas as pd

df = pd.DataFrame(
    {
        'A': ['a', 'b', 'c', 'a', 'b', 'b'],
        'B': [2, 8, 1, 4, 3, 8],
        'C': [102, 98, 107, 104, 115, 87]
    }
)

df

,A,B,C
0,a,2,102
1,b,8,98
2,c,1,107
3,a,4,104
4,b,3,115
5,b,8,87


Agrupe por la columna A y obtenga el valor medio de otras columnas:

In [2]:
df.groupby('A').mean()

,B,C
A,,
a,3.000000,103.0
b,6.333333,100.0
c,1.000000,107.0


#### Agrupando por multiples columnas.

In [3]:
df.groupby(['A', 'B']).mean()

C
A B       
a 2  102.0
  4  104.0
b 3  115.0
  8   92.5
c 1  107.0

>Nota: Observe cómo después de agrupar cada fila en el DataFrame resultante se indexa mediante una **tupla o MultiIndex** (en este caso un par de elementos de las columnas A y B).

Para aplicar varios métodos de agregación a la vez, por ejemplo para contar el número de elementos en cada grupo y calcular su media, use la función *agg*:

In [10]:
df.groupby(['A', 'B']).agg(['count', 'mean'])


C       
    count   mean
A B             
a 2     1  102.0
  4     1  104.0
b 3     1  115.0
  8     2   92.5
c 1     1  107.0

#### Números de agrupación:
Para es siguiente DataFrame:

In [11]:
import numpy as np
np.random.seed(0)

df = pd.DataFrame(
    {
        'Age': np.random.randint(20, 70, 100),
        'Sex': np.random.choice(['Male', 'Female'], 100),
        'number_of_food': np.random.randint(1, 20, 100)
    }
)
df.head()

,Age,Sex,number_of_food
0,64,Female,14
1,67,Female,14
2,20,Female,12
3,23,Male,17
4,23,Female,15


Agrupe la edad en tres categorías (o contenedores). Los contenedores se pueden dar como:

- un número entero $n$ que indica el número de contenedores; en este caso, los datos del marco de datos se dividen en $n$ intervalos de igual tamaño
- una secuencia de números enteros que denota el punto final de los intervalos abiertos en los que se dividen los datos; por ejemplo, `bins=[19, 40, 65, np.inf]` crea tres grupos de edad `(19, 40], (40, 65]` y `(65, np.inf]` 

Pandas asigna automáticamente las versiones de cadena de los intervalos como etiqueta. También es posible definir etiquetas propias definiendo un parámetro de etiquetas como una lista de cadenas.

In [12]:
pd.cut(df['Age'], bins=4) # pd.cut() crea 4 grupos dee edades.


0       (56.75, 69.0]
1       (56.75, 69.0]
2     (19.951, 32.25]
3     (19.951, 32.25]
4     (19.951, 32.25]
           ...       
95      (32.25, 44.5]
96      (32.25, 44.5]
97      (32.25, 44.5]
98      (56.75, 69.0]
99      (56.75, 69.0]
Name: Age, Length: 100, dtype: category
Categories (4, interval[float64, right]): [(19.951, 32.25] < (32.25, 44.5] < (44.5, 56.75] < (56.75, 69.0]]

In [13]:
pd.cut(df['Age'], bins=[19, 40, 65, np.inf])

0     (40.0, 65.0]
1      (65.0, inf]
2     (19.0, 40.0]
3     (19.0, 40.0]
4     (19.0, 40.0]
          ...     
95    (19.0, 40.0]
96    (19.0, 40.0]
97    (40.0, 65.0]
98     (65.0, inf]
99     (65.0, inf]
Name: Age, Length: 100, dtype: category
Categories (3, interval[float64, right]): [(19.0, 40.0] < (40.0, 65.0] < (65.0, inf]]

Úselo en `groupby` para obtener el número medio de foo:

> Nota: Advertencia futura: el valor predeterminado `observado = Falso` está obsoleto y se cambiará a **Verdadero** en una versión futura de pandas. Pase observado=Falso para conservar el comportamiento actual u `observado=Verdadero` para adoptar el valor predeterminado futuro y silenciar esta advertencia.

In [17]:
age_groups = pd.cut(df['Age'], bins=[19, 40, 65, np.inf])
df.groupby(age_groups)['number_of_food'].mean()

/tmp/ipykernel_12359/3744027710.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(age_groups)['number_of_food'].mean()


Age
(19.0, 40.0]    9.880000
(40.0, 65.0]    9.452381
(65.0, inf]     9.250000
Name: number_of_food, dtype: float64

Tabulación cruzada de grupos de edades y género.

In [18]:
pd.crosstab(age_groups, df['Sex'])

Sex,Female,Male
Age,,
"(19.0, 40.0]",22,28
"(40.0, 65.0]",18,24
"(65.0, inf]",3,5


#### Selección de columnas de un grupo.

Cuando haces un `groupby`, puedes seleccionar una sola columna o una lista de columnas:

In [19]:
df = pd.DataFrame(
    [
        [1, 1, 2],
        [1, 2, 3],
        [2, 3, 4]
    ],
    columns=['A', 'B', 'C']
)
df

,A,B,C
0,1,1,2
1,1,2,3
2,2,3,4


In [20]:
g = df.groupby('A')
g['B'].mean() # Solo la columna 'B'

A
1    1.5
2    3.0
Name: B, dtype: float64

In [21]:
g[['B', 'C']].mean() # Las columnas 'B' y 'C'

,B,C
A,,
1,1.5,2.5
2,3.0,4.0


También puedes usar `agg` para especificar columnas y agregaciones para realizar:

In [22]:
g.agg({'B': 'mean', 'C': 'count'})

,B,C
A,,
1,1.5,2
2,3.0,1


#### Agregación por tamaño versus por recuento.
La diferencia entre `size` y `count` es:
el `size` cuenta los valores de `NaN`, el `count` no.

In [24]:
df = pd.DataFrame(
        {
        "Name":["Alice", "Bob", "Mallory", "Mallory", "Bob" , "Mallory"],
        "City":["Seattle", "Seattle", "Portland", "Seattle", "Seattle", "Portland"],
        "Val": [4, 3, 3, np.nan, np.nan, 4]
        }
    )
df

,Name,City,Val
0,Alice,Seattle,4.0
1,Bob,Seattle,3.0
2,Mallory,Portland,3.0
3,Mallory,Seattle,NaN
4,Bob,Seattle,NaN
5,Mallory,Portland,4.0


In [25]:
df.groupby(['Name', 'City'])['Val'].size().reset_index(name='Size')

,Name,City,Size
0,Alice,Seattle,1
1,Bob,Seattle,2
2,Mallory,Portland,2
3,Mallory,Seattle,1


In [26]:
df.groupby(['Name', 'City'])['Val'].count().reset_index(name='Count')

,Name,City,Count
0,Alice,Seattle,1
1,Bob,Seattle,1
2,Mallory,Portland,2
3,Mallory,Seattle,0


#### Agregando grupos.

In [28]:
df = pd.DataFrame(
    {
        'A': list('XYZXYZXYZX'),
        'B': [1,2,1,3,1,2,3,3,1,2],
        'C': [12,14,11,12,13,14,16,12,10,19]
    }
)
df

,A,B,C
0,X,1,12
1,Y,2,14
2,Z,1,11
3,X,3,12
4,Y,1,13
5,Z,2,14
6,X,3,16
7,Y,3,12
8,Z,1,10
9,X,2,19


In [33]:
df.groupby('A')['B'].agg([('mean', np.mean), ('standard deviation', np.std)])

/tmp/ipykernel_12359/3749902042.py:1: FutureWarning: The provided callable <function mean at 0x7f21f8171040> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df.groupby('A')['B'].agg([('mean', np.mean), ('standard deviation', np.std)])
/tmp/ipykernel_12359/3749902042.py:1: FutureWarning: The provided callable <function std at 0x7f21f8171160> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  df.groupby('A')['B'].agg([('mean', np.mean), ('standard deviation', np.std)])


,mean,standard deviation
A,,
X,2.250000,0.957427
Y,2.000000,1.000000
Z,1.333333,0.577350


Para multiples columnas:

In [34]:
df.groupby('A').agg({'B': [np.mean, np.std], 'C': [np.sum, 'count']})

/tmp/ipykernel_12359/1459319034.py:1: FutureWarning: The provided callable <function mean at 0x7f21f8171040> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df.groupby('A').agg({'B': [np.mean, np.std], 'C': [np.sum, 'count']})
/tmp/ipykernel_12359/1459319034.py:1: FutureWarning: The provided callable <function std at 0x7f21f8171160> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  df.groupby('A').agg({'B': [np.mean, np.std], 'C': [np.sum, 'count']})
/tmp/ipykernel_12359/1459319034.py:1: FutureWarning: The provided callable <function sum at 0x7f21f816c0d0> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df.groupby('A').a

B             C      
       mean       std sum count
A                              
X  2.250000  0.957427  59     4
Y  2.000000  1.000000  39     3
Z  1.333333  0.577350  35     3

#### Exportar grupos en diferentes archivos.

Puede iterar sobre el objeto devuelto por `groupby()`. 
El iterador contiene tuplas de`(Categorías, DataFrames)`.

In [ ]:
df = pd.DataFrame(
    {
        'Age': np.random.randint(20, 70, 100),
        'Sex': np.random.choice(['Male', 'Female'], 100),
        'number_of_food': np.random.randint(1, 20, 100)
    }
)
# Export to Male.csv and Female.csv files.
for sex, data in df.groupby('Sex'):
    data.to_csv("{}.csv".format(sex))

#### Usando transform para obtener estadísticas a nivel de grupo mientras se preserva el marco de datos original
ejemplo:

In [35]:
df = pd.DataFrame(
    {
        'group1' : ['A','A','A','A','B','B','B','B'],
        'group2' : ['C','C','C','D','E','E','F','F'],
        'B'     : ['one',np.NaN, np.nan, np.nan,
                   np.nan, 'two', np.nan, np.nan],
        'C'     : [np.nan, 1, np.nan, np.nan,
                   np.nan, np.nan, np.nan, 4],

})
df

,group1,group2,B,C
0,A,C,one,NaN
1,A,C,NaN,1.0
2,A,C,NaN,NaN
3,A,D,NaN,NaN
4,B,E,NaN,NaN
5,B,E,two,NaN
6,B,F,NaN,NaN
7,B,F,NaN,4.0


Quiero obtener el recuento de observaciones no faltantes de B para cada combinación de grupo1 y grupo2. `groupby.transform` es una función muy poderosa que hace exactamente eso.

In [37]:
df['count_B']=df.groupby(['group1','group2'])['B'].transform('count')
df

,group1,group2,B,C,count_B
0,A,C,one,NaN,1
1,A,C,NaN,1.0,1
2,A,C,NaN,NaN,1
3,A,D,NaN,NaN,0
4,B,E,NaN,NaN,1
5,B,E,two,NaN,1
6,B,F,NaN,NaN,0
7,B,F,NaN,4.0,0


Read Grouping Data [online:](https://riptutorial.com/pandas/topic/1822/grouping-data)